In [1]:
from rdflib import Graph

In [2]:
#Work
uri = 'http://id.loc.gov/resources/works/748300.bibframe.rdf'
g = Graph()
g.parse(uri)
g.serialize('oldman.nt')

<Graph identifier=N56a8ffcc8ceb4561870d59ba1a29ca17 (<class 'rdflib.graph.Graph'>)>

In [ ]:
#Instancia
uri = 'http://id.loc.gov/resources/instances/1893399.bibframe.rdf'
g = Graph()
g.parse(uri)

In [ ]:
#Item
uri = 'http://id.loc.gov/resources/items/748300-050-10.bibframe.rdf'
g = Graph()
g.parse(uri)
g.serialize('item.nt')

In [4]:
#Subject
uri = 'http://id.loc.gov/authorities/subjects/sh85113021.rdf'
g = Graph()
g.parse(uri)
g.serialize('Research.nt')

<Graph identifier=N185bf5b0cdb943e19185b998a13daef1 (<class 'rdflib.graph.Graph'>)>

In [2]:
from pyfuseki import FusekiQuery

In [3]:
fuseki_query = FusekiQuery('http://localhost:3030', 'thesaurus')

GRAPH EXISTS

In [4]:
query = """PREFIX bk: <https://bibliokeia.com/authorities/subjects/>
  ASK WHERE { GRAPH bk:sh85066157 { ?s ?p ?o } }"""

query_result = fuseki_query.run_sparql(query)
query_result

In [10]:
for i in query_result:
    print(i)

b'{ \n'
b'  "head" : { } ,\n'
b'  "boolean" : false\n'
b'}\n'


In [4]:
query = """
    PREFIX madsrdf:  <http://www.loc.gov/mads/rdf/v1#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX bk: <https://bibliokeia.com/authorities/subjects/>
SELECT (COUNT(*) AS ?count)
WHERE { GRAPH  bk:sh85084414
      {?s ?p ?o . }
    }
"""

query_result = fuseki_query.run_sparql(query)
query_result

In [ ]:
query_result

In [28]:
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore

In [29]:
store = SPARQLUpdateStore(update_endpoint='http://localhost:3030/thesaurus/update')
query_endpoint = 'http://localhost:3030/thesaurus/query'
update_endpoint = 'http://localhost:3030/thesaurus/update'
store.open((query_endpoint, update_endpoint))

In [14]:
response = store.query(query)

In [16]:
response.askAnswer

False

In [17]:
def GraphExist(token):
    store = SPARQLUpdateStore(update_endpoint='http://localhost:3030/thesaurus/update')
    query_endpoint = 'http://localhost:3030/thesaurus/query'
    update_endpoint = 'http://localhost:3030/thesaurus/update'
    store.open((query_endpoint, update_endpoint))

    query = "PREFIX bk: <https://bibliokeia.com/authorities/subjects/>\n \
                ASK WHERE { GRAPH bk:" + token +" { ?s ?p ?o } }"
    
    response = store.query(query)

    return response.askAnswer
    


    

In [19]:

def UpdateThesarus(graph, metadata, token):

    store = SPARQLUpdateStore(update_endpoint='http://localhost:3030/thesaurus/update')
    query_endpoint = 'http://localhost:3030/thesaurus/query'
    update_endpoint = 'http://localhost:3030/thesaurus/update'
    store.open((query_endpoint, update_endpoint))

    prefix = "PREFIX madsrdf:  <http://www.loc.gov/mads/rdf/v1#>\n \
        PREFIX bk: <https://bibliokeia.com/authorities/subjects/>\n \
        PREFIX lc: <http://id.loc.gov/authorities/subjects/>\n"

    q1 = prefix + "WITH bk:" + graph + "\n \
        DELETE {?s madsrdf:" + metadata + " lc:" + token + "}\n \
        INSERT { ?s madsrdf:" + metadata + " bk:" + token + "}\n \
        WHERE { ?s madsrdf:" + metadata + " lc:" + token + "}\n"
    store.update(q1)

    q2 = prefix + "WITH bk:" + graph + "\n \
            DELETE { lc:" + metadata + " ?p ?o }\n \
                INSERT { bk:" + metadata + " ?p ?o }\n \
                    WHERE { lc:" + metadata + "?p ?o }"
    store.update(q2)

    q3 = prefix + "WITH bk:" + graph + "\n \
            DELETE { lc:" + metadata + " madsrdf:isMemberOfMADSCollection ?o }\n \
                INSERT { bk:" + metadata + " madsrdf:isMemberOfMADSCollection bk:collection_BKSH_General }\n \
                    WHERE { lc:" + metadata + " madsrdf:isMemberOfMADSCollection ?o }"
    store.update(q3)

False

In [27]:
prefix = "PREFIX madsrdf:  <http://www.loc.gov/mads/rdf/v1#>\n \
        PREFIX bk: <https://bibliokeia.com/authorities/subjects/>\n \
        PREFIX lc: <http://id.loc.gov/authorities/subjects/>\n"

In [30]:
graph = "sh85084414"
token = "sh85113021"
q2 = prefix + "WITH bk:" + graph + "\n \
            DELETE { bk:sh85113021 ?p lc:collection_LCSH_General }\n \
                INSERT { bk:sh85113021 ?p bk:collection_BKSH_General}\n \
                    WHERE { bk:sh85113021 ?p lc:collection_LCSH_General  }"
store.update(q2)


In [ ]:
q1 = prefix + "WITH bk:" + graph + "\n \
        DELETE {?s madsrdf:" + metadata + " lc:" + token + "}\n \
        INSERT { ?s madsrdf:" + metadata + " bk:" + token + "}\n \
        WHERE { ?s madsrdf:" + metadata + " lc:" + token + "}\n"

In [25]:
q3 = prefix + "WITH bk:sh85084414 \n \
            DELETE { bk:sh85113021 madsrdf:isMemberOfMADSCollection ?o }\n \
                INSERT { bk:sh85113021 madsrdf:isMemberOfMADSCollection bk:collection_BKSH_General }\n \
                    WHERE { bk:sh85113021 madsrdf:isMemberOfMADSCollection ?o }"
store.update(q3)

HTTPError: HTTP Error 400: Bad Request